<a href="https://colab.research.google.com/github/GenAIHub/genai-workshop/blob/main/02_RAG/01_data_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install boto3
!pip install pydantic==1.10.9
!pip installl lama_index
!pip install llama-index-embeddings-bedrock==0.1.4
!pip install llama-index-llms-bedrock==0.1.5
!pip install llama-index-vector-stores-postgres==0.1.5
!pip install langchain==0.0.333
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.8.2
    Uninstalling pydantic-2.8.2:
      Successfully uninstalled pydantic-2.8.2
ERROR: unknown command "installl" - maybe you meant "install"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58

In [2]:
!pip install llama-index-readers-file

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 7.4 MB/s eta 0:00:00


In [3]:
import os
import requests

# Create the data directory if it doesn't exist
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

# Define the URL and the local file path
pdf_url = "https://d687lz8k56fia.cloudfront.net/sec-edgar-filings/0001018724/10-K/0001018724-23-000004/filing-details.pdf"
pdf_path = os.path.join(data_dir, "AMZN.pdf")

# Download the PDF file
response = requests.get(pdf_url)
with open(pdf_path, "wb") as file:
    file.write(response.content)

In [15]:
import fitz

doc = fitz.open(pdf_path)
# doc = fitz.open("./data/your_file.pdf")

# Print the number of pages in the PDF
print(f"\nThe document has {doc.page_count} pages.\n")

full_pdf_content = ""
for page in range(doc.page_count):
  pg = doc.load_page(page)
  text = pg.get_text()
  full_pdf_content += text

print(f"\n{full_pdf_content[:1000]}")


The document has 75 pages.


Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
 ____________________________________
FORM 10-K
____________________________________ 
(Mark One)
☒
ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended December 31, 2022
or
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from            to             .
Commission File No. 000-22513
____________________________________
AMAZON.COM, INC.
(Exact name of registrant as specified in its charter)
Delaware
 
91-1646860
(State or other jurisdiction of
incorporation or organization)
 
(I.R.S. Employer
Identification No.)
410 Terry Avenue North
Seattle, Washington 98109-5210
(206) 266-1000
(Address and telephone number, including area code, of registrant’s principal executive offices)
Securities registered pursuant to Section 12(b) of the Act:
Title of E

In [5]:
from llama_index.core.node_parser.text.sentence import SentenceSplitter
from llama_index.core.schema import Document
import pprint as pp

chunk_size = 1024
chunk_overlap = 100

splitter = SentenceSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

docs = splitter.get_nodes_from_documents(
        [
            Document(
                text=full_pdf_content,
                metadata={
                    "page": 0,
                    #"file_name": f"{metadata_file_name}",
                    "source": "text",
                    "document_id": f"some_id",
                },
                excluded_embed_metadata_keys=["page", "source", "document_id"],
                excluded_llm_metadata_keys=["page", "source", "document_id"],
            )
        ]
    )
pp.pprint(docs[0])

TextNode(id_='b07881b7-8b55-4a22-ac2b-0d4500c94c5a', embedding=None, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id'}, excluded_embed_metadata_keys=['page', 'source', 'document_id'], excluded_llm_metadata_keys=['page', 'source', 'document_id'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd83b0bf-ffdb-406f-9832-f1ea75edbdc9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id'}, hash='98b9dc512b239b5301a6757daafca0dcec035a1e353aa95f9e7dbb3bd3200805'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='a15f28a2-15d6-4947-abb2-15efcac07f4b', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='65e87666bf7d22de2a1fee8c14b3fa9e438711ac416d8c241b45fef710d11fc0')}, text='Table of Contents\nUNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n ____________________________________\nFORM 10-K\n____________________________________ \n(Mark One)\n☒\nANNUAL REPORT PURSUANT TO SECT

In [6]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.llms.bedrock import Bedrock
from llama_index.core.settings import Settings
import boto3
import asyncio
import os


os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""


embed_model_id = "amazon.titan-embed-text-v1"
llm_model_id = "anthropic.claude-3-haiku-20240307-v1:0"

BEDROCK_CLIENT = boto3.client("bedrock-runtime", "us-east-1")

llm = Bedrock(
    model=llm_model_id, client=BEDROCK_CLIENT, aws_region_name="us-east-1")
embed_model = BedrockEmbedding(model=embed_model_id, client=BEDROCK_CLIENT)

Settings.llm = llm
Settings.embed_model = embed_model

In [7]:
from llama_index.core.ingestion.pipeline import IngestionPipeline
from llama_index.core.extractors import QuestionsAnsweredExtractor
import asyncio

pipeline = IngestionPipeline(
          transformations=[
              QuestionsAnsweredExtractor(questions=2)
           ]
      )
nodes = await pipeline.arun(nodes = docs)
pp.pprint(nodes[0])

100%|██████████| 67/67 [03:45<00:00,  3.37s/it]  

TextNode(id_='b07881b7-8b55-4a22-ac2b-0d4500c94c5a', embedding=None, metadata={'page': 0, 'source': 'text', 'document_id': 'some_id', 'questions_this_excerpt_can_answer': 'Based on the provided context, here are two questions that can be answered using the information given:\n\n1. What is the ticker symbol for Amazon.com, Inc.\'s common stock?\nThe context states that the "Title of Each Class" for Amazon\'s securities registered pursuant to Section 12(b) of the Act is "Common Stock, par value $.01 per share" and the "Trading Symbol(s)" is "AMZN".\n\n2. What was the aggregate market value of the voting stock held by non-affiliates of Amazon.com, Inc. as of June 30, 2022?\nThe context provides this specific piece of information, stating that the "Aggregate market value of voting stock held by non-affiliates of the registrant as of June 30, 2022" was "$944,744,113,598".\n\nThe context provided is the beginning of Amazon.com, Inc.\'s Form 10-K annual report filing with the U.S. Securities 

In [8]:
from llama_index.core import VectorStoreIndex

index =  VectorStoreIndex(nodes)

print(index)

In [9]:
query_engine = index.as_query_engine(similarity_top_k=2)
response = query_engine.query("What is this document about?")

print(response)

This document appears to be discussing the legal, regulatory, and operational risks and challenges faced by a large technology company, likely Amazon. The key points covered in the document include:

1. Legal and regulatory risks:
- Evolving government regulations and laws governing various aspects of the company's operations, including the internet, e-commerce, digital content, web services, advertising, AI, etc.
- Formal and informal reviews, investigations, and proceedings by government and regulatory authorities, including competition-related investigations.
- Potential for unfavorable regulations, laws, decisions, or interpretations that could increase costs, change business practices, damage reputation, or impact operations.

2. Claims, litigation, and other proceedings:
- The company is regularly subject to a wide range of claims, litigation, and other proceedings, including those related to intellectual property, taxes, labor, competition/antitrust, privacy, consumer protection

In [10]:
for n in response.source_nodes:
  print(n)

Node ID: aca1ef50-6fcc-4ef3-a55d-d4ee4a58cf16
Text: Volatility in our stock price could adversely affect our
business and financing opportunities and force us to increase our cash
compensation to employees or grant larger stock awards than we have
historically, which could hurt our operating results or reduce the
percentage ownership of our existing stockholders, or both. Legal and
Regulatory Ris...
Score:  0.305

Node ID: d5575e10-c145-42b8-9c0d-b9df0b605c58
Text: When these policies and processes are circumvented or fail to
operate sufficiently, it can harm our business or damage our
reputation and we could face civil or criminal liability for unlawful
activities by our sellers. Under our A-to-z Guarantee, we may
reimburse customers for payments up to certain limits in these
situations, and as our third-...
Score:  0.270

